In [1]:
import json
import os
import re
import pandas as pd

# Directory with JSON files
input_dir = "../output"

models = [
    "meta-llama/Llama-2-7b-hf",
    "mistralai/Mistral-7B-v0.3",
]


output_csv = os.path.join(input_dir, "metrics.csv")

filename_pattern = re.compile(
    r"output_(?:align=(\d+)_)?(?:delta=\d+(?:\.\d+)?_gamma=\d+(?:\.\d+)?_k=\d+_)?seed=(\d+)_watermark=(mb2|mb|gaussmark)(?:_dataset=([a-zA-Z0-9_\-]+))?\.json"
)

rows = []

for model in models:
    model_suffix = model.split("/")[-1]
    model_dir = os.path.join(input_dir, model_suffix)
    json_files = [f for f in os.listdir(model_dir) if f.endswith(".json")]
    for json_file in json_files:
        match = filename_pattern.match(json_file)
        if not match:
            continue

        align = int(match.group(1)) if match.group(1) is not None else None
        dataset = match.group(4) if match.group(4) is not None else None
        filepath = os.path.join(model_dir, json_file)

        with open(filepath, "r") as f:
            data = json.load(f)

            config = data["config"]
            metrics = data["metrics"]
            metrics_60 = data.get("metrics_dipper_text_lex60_order0", {})
            metrics_20 = data.get("metrics_dipper_text_lex20_order0", {})
            ppl = data.get("ppl", {})
            watermark = data["watermark"]

            if watermark == "gaussmark":
                config_id = f"watermark=gaussmark_sigma=0.04"
            elif watermark == "mb":
                config_id = f"align={align}_gamma={config['gamma']}_delta={config['delta']}_nclusters={config['n_clusters']}"
            elif watermark == "mb2":
                config_id = f"watermark=mb2"
            

            row = {
                "config_id": config_id,
                "dataset": dataset,
                "n_clusters": config.get("n_clusters"),
                "auroc": metrics.get("auroc"),
                "best_f1_score": metrics.get("best_f1_score"),
                "tpr_1_fpr": metrics.get("tpr_1_fpr"),
                "tpr_0.1_fpr": metrics.get("tpr_0.1_fpr"),
                "auroc_lex60": metrics_60.get("auroc"),
                "f1_lex60": metrics_60.get("best_f1_score"),
                "tpr1_lex60": metrics_60.get("tpr_1_fpr"),
                "auroc_lex20": metrics_20.get("auroc"),
                "f1_lex20": metrics_20.get("best_f1_score"),
                "tpr1_lex20": metrics_20.get("tpr_1_fpr"),
                "ppl_mean": ppl.get("mean"),
                "mean_seq_rep_3": data.get("mean_seq_rep_3"),
                "model": model_suffix,
            }

            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

# Group by config_id and average all metric columns
grouped_df = df.groupby(["config_id", "dataset", "model"]).mean(
    numeric_only=True).reset_index()

# Write to CSV
grouped_df.to_csv(output_csv, index=False)
print(f"Saved grouped averages to: {output_csv}")

Saved grouped averages to: ../output/metrics.csv
